In [2]:
# 构建一个玩具数据集
corpus = [
    "我喜欢吃苹果",
    "我喜欢吃香蕉",
    "她喜欢吃葡萄",
    "他不喜欢吃香蕉",
    "他喜欢吃苹果",
    "她喜欢吃草莓"
]

In [3]:
# 定义一个分词函数，将文本转化为单个字符的列表
def tokenize(text):
    return [char for char in text]  # 将文本拆分为字符列表

In [9]:
# 定义计算N-Gram词频的函数
from collections import defaultdict, Counter  # 导入所需的库
def count_ngrams(corpus, n):
    ngrams_count = defaultdict(Counter)  # 创建一个嵌套词典，存储N-Gram计数
    for text in corpus:  # 遍历语料库中的每一个文本
        tokens = tokenize(text)  # 对文本进行分词
        for i in range(len(tokens) - n + 1):  # 遍历文本中的每一个N-Gram
            ngram = tuple(tokens[i:i + n])  # 获取当前N-Gram
            prefix = ngram[:-1]  # 获取当前N-Gram的前缀
            token = ngram[-1]  # 获取当前N-Gram的目标单字
            ngrams_count[prefix][token] += 1  # 更新N-Gram计数
    return ngrams_count

bigram_counts = count_ngrams(corpus, 2)  # 计算二元语法的词频
print("Bigram 词频：")
for prefix, counter in bigram_counts.items():
    print(prefix, ":", counter)

Bigram 词频：
('我',) : Counter({'喜': 2})
('喜',) : Counter({'欢': 6})
('欢',) : Counter({'吃': 6})
('吃',) : Counter({'苹': 2, '香': 2, '葡': 1, '草': 1})
('苹',) : Counter({'果': 2})
('香',) : Counter({'蕉': 2})
('她',) : Counter({'喜': 2})
('葡',) : Counter({'萄': 1})
('他',) : Counter({'不': 1, '喜': 1})
('不',) : Counter({'喜': 1})
('草',) : Counter({'莓': 1})


In [14]:
# 定义计算N-Gram出现概率的函数
def ngram_probabilities(ngram_counts):
    ngram_probs = defaultdict(Counter)  # 创建一个嵌套词典，存储N-Gram概率
    for prefix, tokens_count in ngram_counts.items(): # 遍历N-Gram前缀
        total_count = sum(tokens_count.values())  # 计算当前前缀的总词频
        for token, count in tokens_count.items(): # 遍历当前前缀的所有目标单字
            ngram_probs[prefix][token] = count / total_count # 计算当前N-Gram的概率
    return ngram_probs

bigram_probs = ngram_probabilities(bigram_counts)  # 计算二元语法的概率
print("bigram出现的概率：")
for prefix, probs in bigram_probs.items():
    print("{}: {}".format("".join(prefix), dict(probs)))

bigram出现的概率：
我: {'喜': 1.0}
喜: {'欢': 1.0}
欢: {'吃': 1.0}
吃: {'苹': 0.3333333333333333, '香': 0.3333333333333333, '葡': 0.16666666666666666, '草': 0.16666666666666666}
苹: {'果': 1.0}
香: {'蕉': 1.0}
她: {'喜': 1.0}
葡: {'萄': 1.0}
他: {'不': 0.5, '喜': 0.5}
不: {'喜': 1.0}
草: {'莓': 1.0}


In [15]:
# 定义生成下一个词的函数
def generate_next_token(prefix, ngram_probs):
    if not prefix in ngram_probs:  # 如果前缀不在N-Gram概率词典中，返回None
        return None
    next_token_probs = ngram_probs[prefix]  # 获取当前前缀的所有目标单字概率
    next_token = max(next_token_probs, key=next_token_probs.get)  # 获取概率最大的目标单字
    return next_token

In [16]:
# 定义生成连续文本的函数
def generate_text(prefix, ngram_probs, n, length=6):
    tokens = list(prefix) # 将前缀转化为列表
    for _ in range(length - len(prefix)):
        # 获取当前前缀的下一个词
        next_token = generate_next_token(tuple(tokens[-n+1:]), ngram_probs)
        if not next_token: # 如果下一个词不存在，结束生成
            break
        tokens.append(next_token)  # 将下一个词添加到文本中
    return "".join(tokens)  # 将生成的文本转化为字符串

In [18]:
# 生成文本
generated_text = generate_text("她", bigram_probs, 2)
print("生成的文本：", generated_text)

生成的文本： 她喜欢吃苹果
